::: {.cell .markdown}

In [ ]:
#| echo: false
#| output: 'asis'
#| include: true
#| eval: true

import sys
sys.path.insert(1, '../../../../') #insert the utils module
from utils import print_badges

#print_badges(__file__)
print_badges("content/course/manipulation/03_geopandas_tutorial.qmd")

:::

Dans ce tutoriel, nous allons utiliser les données suivantes :

* [Localisations des stations velib](https://opendata.paris.fr/explore/dataset/velib-emplacement-des-stations/download/?format=geojson&timezone=Europe/Berlin&lang=fr) ;
* [fonds de carte `AdminExpress`](https://geoservices.ign.fr/adminexpress) à
travers un package `Python` nommé [`cartiflette`](https://github.com/InseeFrLab/cartogether)
facilitant la récupération de cette source. 

La représentation des données, notamment la cartographie, est présentée plus
amplement dans la partie [visualiser](#visualiser). Quelques méthodes
pour faire rapidement des cartes seront présentées ici, mais
l'objet de ce chapitre porte davantage sur la manipulation des données géographiques.

Ce tutoriel s'inspire beaucoup d'un autre tutoriel que j'ai fait pour
`R` disponible
[dans la documentation `utilitr`](https://www.book.utilitr.org/spatdata.html).
Il peut servir de pendant à celui-ci pour l'utilisateur de `R`. 


Quelques installations préalables sont nécessaires : 

```python
!pip install pandas fiona shapely pyproj rtree # à faire obligatoirement en premier pour utiliser rtree ou pygeos pour les jointures spatiales
!pip install contextily
!pip install geopandas
!pip install topojson
```


In [ ]:
#| echo: false
!pip install topojson

Pour être en mesure d'exécuter ce tutoriel, les imports suivants
seront utiles.


In [ ]:
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt

# Données spatiales: quelle différence avec des données traditionnelles ?

**Le terme "données spatiales" désigne les données qui portent sur les caractéristiques géographiques des objets (localisation, contours, liens)**.
Les caractéristiques géographiques des objets sont décrites à l'aide d'un **système de coordonnées**
qui permettent une représentation dans un espace euclidien ($(x,y)$).
Le passage de l'espace réel (la Terre, qui est une sphère) à l'espace plan
se fait grâce à un **système de projection**. Voici quelques exemples
de données spatiales : 

* Une table décrivant des bâtiments, avec les coordonnées géographiques de chaque bâtiment;
* Le découpage communal du territoire, avec le contour du territoire de chaque commune;
* Les routes terrestres, avec les coordonnées décrivant leur parcours.

Les données spatiales rassemblent classiquement deux types de données :

1. des **données géographiques** (ou géométries): objets géométriques tels que des points, des vecteurs, des polygones, ou des maillages (*raster*). Exemple: la forme de chaque chaque commune, les coordonnées d'un bâtiment;
2. des **données attributaires** (ou attributs): des mesures et des caractéristiques associés aux objets géométriques. Exemple: la population de chaque commune, le nombre de fenêtres et le nombre d'étages d'un bâtiment.

**Les données spatiales sont fréquemment traitées à l'aide d'un système d'information géographique (SIG)**, c'est-à-dire un système d'information capable de stocker, d'organiser et de présenter des données alphanumériques spatialement référencées par des coordonnées dans un système de référence (CRS). `Python` dispose de fonctionnalités lui permettant de réaliser les mêmes tâches qu'un SIG (traitement de données spatiales, représentations cartographiques).

**Les systèmes de projection font l'objet de standards internationaux et sont souvent désignés par des codes dits codes EPSG**. Ce [site](https://epsg.io/) est un bon aide-mémoire. Les plus fréquents, pour les utilisateurs français, sont les suivants (plus d'infos [ici](https://geodesie.ign.fr/contenu/fichiers/documentation/SRCfrance.pdf)):

* `2154`: système de projection Lambert 93. Il s'agit du système de projection officiel. La plupart des données diffusées par l'administration pour la métropole sont disponibles dans ce système de projection. 
* `27572`: Lambert II étendu. Il s'agit de l'ancien système de projection officiel. Les données spatiales anciennes peuvent être dans ce format.
* `4326`: WGS 84 ou système de pseudo-Mercator. Attention, ce n'est en réalité pas un système de projection mais un système de coordonnées (longitude / latitude) qui permet simplement un repérage angulaire sur l'ellipsoïde. Il est utilisé pour les données GPS.

## De `pandas` à `geopandas`

Le *package* `geopandas` est une boîte à outils conçue pour faciliter la manipulation de données spatiales. **La grande force de `geopandas` est qu'il permet de manipuler des données spatiales comme s'il s'agissait de données traditionnelles**, car il repose sur le standard ISO 19125 [*simple feature access*](https://en.wikipedia.org/wiki/Simple_Features) défini conjointement par l'*Open Geospatial Consortium (OGC)* et l'*International Organization for Standardization (ISO)*. 

Par rapport à un DataFrame standard, un objet `geopandas` comporte
une colonne supplémentaire: `geometry`. Elle stocke les coordonnées des
objets géographiques (ou ensemble de coordonnées s'agissant de contours). Un objet `geopandas` hérite des propriétés d'un 
DataFrame pandas mais propose des méthodes adaptées au traitement des données spatiales. 

Ainsi, grâce à  `Geopandas`, on pourra effectuer des manipulations sur les attributs des données comme avec `pandas` mais on pourra également faire des manipulations sur la dimension spatiale des données. En particulier,

* Calculer des distances et des surfaces;
* Agréger rapidement des zonages (regrouper les communes en département par exemple);
* Trouver dans quelle commune se trouve un bâtiment à partir de ses coordonnées géographiques;
* Recalculer des coordonnées dans un autre système de projection.
* Faire une carte, rapidement et simplement

::: {.cell .markdown}

```{=html}
<div class="alert alert-warning" role="alert">
<h3 class="alert-heading"><i class="fa-solid fa-lightbulb"></i> Hint</h3>
```


Les manipulations de données sur un objet `Geopandas` sont nettement plus lentes que sur
un `DataFrame` traditionnel (car `Python` doit gérer les informations géographiques pendant la manipulation des données).
Lorsque vous manipulez des données de grandes dimensions,
il peut être préférable d’effectuer les opérations sur les données avant de joindre une géométrie à celles-ci.


```{=html}
</div>
```

:::

Par rapport à un logiciel spécialisé comme `QGIS`, `Python` permettra 
d'automatiser le traitement et la représentation des données. D'ailleurs,
`QGIS` utilise lui-même `python`...


# Importer des données spatiales

Les données spatiales sont plus riches que les données traditionnelles car elles
incluent, habituellement, des éléments supplémentaires pour placer dans
un espace cartésien les objets. Cette dimension supplémentaire peut être simple
(un point comporte deux informations supplémentaire: $x$ et $y$) ou
assez complexe (polygones, lignes avec direction, etc.)

Les formats les plus communs de données spatiales sont les suivants :

* _shapefile_ (`.shp`): format (**propriétaire**) le plus commun de données géographiques.
La table de données (attributs) est stockée dans un fichier séparé des
données spatiales. En faisant `geopandas.read_file("monfichier.shp")`, le
package fait lui-même le lien entre les observations et leur représentation spatiale ;
* _geopackage_ (`.gpkg`) : ce (relativement) nouveau format **libre** en un seul fichier également (lui recommandé par l'OGC) vise progressivement à se substituer au shapefile. Il est par exemple le format par défaut dans QGIS. 
* _geojson_ (`.json`) : ce format, non préconisé par l'OGC est largement utilisé pour le développement web
comme dans la librairie `leaflet.js`.
La dimension spatiale est stockée dans le même fichier que les attributs.
Ces fichiers sont généralement beaucoup plus légers que les *shapefiles* mais possèdent des limites s'agissant de gros jeux de données. 

Cette [page](https://si.ecrins-parcnational.com/blog/2020-02-geojson-shapefile-geopackage.html) compare plus en détail ces trois types de formats de données géographiques. 

L'aide de [geopandas](https://geopandas.org/io.html) propose des bouts de code en fonction des différentes situations dans lesquelles on se trouve.

## Exemple: récupérer les découpages territoriaux

L'un des fonds de carte les plus fréquents qu'on utilise est celui des
limites administratives des communes.
Celui-ci peut être récupéré de plusieurs manières. 
En premier lieu, pour récupérer 
le fond de carte officiel, produit par l'IGN, sous 
le nom d'[`AdminExpress`](https://geoservices.ign.fr/adminexpress#telechargementCog)[^1],
il est possible de se rendre sur le site de l'IGN et de le télécharger depuis
le serveur `FTP`. 
Il est également possible d'utiliser l'une des [API de l'IGN](https://api.gouv.fr/les-api/api_carto_cadastre)
mais ces dernières ne sont pas encore très documentées pour des utilisateurs
de `Python`.
Le package `pynsee` propose notamment un module dédié à la récupération de fonds de carte officiels pour valoriser des données
d'open data. L'API sur laquelle il repose étant parfois lente, nous présentons le code dédié uniquement en annexe. 


[^1]: Il existe également une version moins officielle sur [data.gouv](https://www.data.gouv.fr/fr/datasets/decoupage-administratif-communal-francais-issu-d-openstreetmap/), construite à 
partir d'`OpenStreetMap`.


Nous proposons ici une méthode nouvelle de récupération de
ces données qui s'appuie sur le projet interministériel 
[`cartiflette`](https://10pourcent.etalab.studio/projets/insee/).  
Ce projet vise à faciliter la récupération des sources officielles, notamment
celles de l'IGN, et leur association à des jeux de données géographiques.

::: {.cell .markdown}

```{=html}
<div class="alert alert-info" role="alert">
<h3 class="alert-heading"><i class="fa-solid fa-comment"></i> Note</h3>
```


Le package `cartiflette` est expérimental 
et n'est disponible que sur
[`Github`](https://github.com/InseeFrLab/cartogether), pas sur `PyPi`.
Il est amené à évoluer rapidement et cette page sera mise à jour
quand de nouvelles fonctionalités (notamment l'utilisation d'`API`)
seront disponibles pour encore simplifier la récupération de
contours géographiques.

Pour installer `cartiflette`, il est nécessaire d'utiliser les commandes suivantes
depuis un `Jupyter Notebook` (si vous utilisez la ligne de commande directement,
vous pouvez retirer les `!` et `%` en début de ligne):


In [ ]:
#| eval: false
!pip install --upgrade botocore==1.23.26  #Sur colab, sinon bug
!pip install --upgrade urllib3==1.22.0 #Sur colab, sinon bug
!pip install py7zr #Sur colab, sinon bug
!pip install s3fs #Sur colab, sinon bug
!git clone https://github.com/InseeFrLab/cartogether.git
%cd ./cartogether
!pip install -r requirements.txt
!pip install .

Ces commandes permettent de récupérer l'ensemble du code 
source depuis [`Github` <i class="fab fa-github"></i></a>](https://github.com/InseeFrLab/cartiflette)


```{=html}
</div>
```

:::


Ici, nous sommes intéressés par les contours des communes 
de la petite couronne. On pourrait désirer récupérer
l'ensemble de la région Ile-de-France mais nous
allons nous contenter de l'analyse de Paris intra-muros
et des départements limitrophes. 
C'est l'un des avantage de `cartiflette` que de faciliter
la récupération de fonds de carte sur un ensemble de département.
Cela évite la récupération d'un fond de carte très
volumineux (plus de 500Mo) pour une analyse restreinte (quelques départements).
Un autre avantage de `cartiflette` est de faciliter la récupération de fonds
de carte consolidés comme celui dont on a besoin ici: arrondissements 
dans Paris, communes ailleurs. Comme cela est expliqué dans un encadré à part,
il s'agirait d'une opération pénible à mettre en oeuvre sans `cartiflette`. 

Les contours de cet espace peuvent être récupérés de la manière suivante:


In [ ]:
import cartiflette.s3 as s3

shp_communes = s3.download_vectorfile_url_all(
    crs = 4326,
    values = ["75", "92", "93", "94"],
    borders="COMMUNE",
    vectorfile_format="topojson",
    filter_by="DEPARTEMENT",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022)

shp_communes.head()

On reconnaît la structure d'un `DataFrame` `Pandas`. A cette structure s'ajoute 
une colonne `geometry` qui enregistre la position des limites des polygones de chaque
observation. 

Comme vu précédemment, le système de projection est un élément important. Il permet à `Python`
d'interpréter les valeurs des points (deux dimensions) en position sur 
la terre, qui n'est pas un espace plan.


In [ ]:
shp_communes.crs

Ici, les données sont dans le système WGS84 (code EPSG 4326).
Ce n'est pas le
Lambert-93 comme on pourrait s'y attendre, ce dernier
étant le système légal de projection pour la France
métropolitaine. 

Pour s'assurer qu'on a bien récupéré les contours voulus, 
on peut représenter graphiquement
les contours grâce à la méthode `plot` sur laquelle nous
reviendrons :


In [ ]:
ax = shp_communes.boundary.plot()
ax.set_axis_off()

::: {.cell .markdown}

```{=html}
<div class="alert alert-success" role="alert">
<h3 class="alert-heading"><i class="fa-solid fa-pencil"></i> Note</h3>
```


Si on ne désire pas utiliser le niveau `COMMUNE_ARRONDISSEMENT`,
il est nécessaire de mettre en oeuvre une construction du fond de
carte en plusieurs phases.

En premier lieu, il est nécessaire de récupérer le niveau des communes. 


In [ ]:
import cartiflette.s3 as s3

shp_communes = s3.download_vectorfile_url_all(
    crs = 4326,
    values = ["75", "92", "93", "94"],
    borders="COMMUNE",
    vectorfile_format="topojson",
    filter_by="DEPARTEMENT",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022)

shp_communes.head()

In [ ]:
ax = shp_communes.boundary.plot()
ax.set_axis_off()

On peut remarquer que la ville de Paris ne comporte pas d'arrondissements
sur cette carte. Pour vous en convaincre, vous pouvez exécuter la 
commande :


In [ ]:
ax = shp_communes.loc[shp_communes['INSEE_DEP']=="75"].boundary.plot()
ax.set_axis_off()

Il faut donc utiliser une source complémentaire.
Le contour officiel des arrondissements est
produit par l'IGN séparemment des contours de communes.
Les contours d'arrondissements sont également
disponibles 
grâce à `cartiflette`:


In [ ]:
arrondissements = s3.download_vectorfile_url_all(
    crs = 4326,
    values = ["75"],
    borders="COMMUNE",
    vectorfile_format="topojson",
    filter_by="DEPARTEMENT",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022)

In [ ]:
ax = arrondissements.plot(alpha = 0.8, edgecolor = "k")
ax.set_axis_off()

Il ne reste plus qu'à remplacer Paris par
ses arrondissements dans `shp_communes`. 
Pour cela, on peut utiliser les méthodes
vues dans le chapitre `Pandas` relatives
aux filtres et à la concaténation
de plusieurs `DataFrames`:


In [ ]:
#| echo: true
import pandas as pd

shp_communes = pd.concat(
  [
    shp_communes.loc[shp_communes['INSEE_DEP'] != "75"].to_crs(2154),
    arrondissements.to_crs(2154)
  ])

Cette approche fonctionne mais elle nécessite un certain nombre
de gestes, qui sont autant de risques d'erreurs. Il est
donc recommandé de privilégier le niveau `COMMUNE_ARRONDISSEMENT`
qui fait exactement ceci mais de manière fiable. 


```{=html}
</div>
```

:::

# Opérations sur les attributs et les géométries

## Import des données velib

Souvent, le découpage communal ne sert qu'en fond de cartes, pour donner des
repères. En complément de celui-ci, on peut désirer exploiter
un autre jeu de données. On va partir des données de localisation des
stations velib, 
disponibles [sur le site d'open data de la ville de Paris](https://opendata.paris.fr/explore/dataset/velib-emplacement-des-stations/table/) et 
requêtables directement par l'url
<https://opendata.paris.fr/explore/dataset/velib-emplacement-des-stations/download/?format=geojson&timezone=Europe/Berlin&lang=fr>


In [ ]:
#| echo: true

velib_data = 'https://opendata.paris.fr/explore/dataset/velib-emplacement-des-stations/download/?format=geojson&timezone=Europe/Berlin&lang=fr'
stations = gpd.read_file(velib_data)
stations.crs

Les données sont dans le système de projection WGS84 qui est celui du
système GPS. Celui-ci s'intègre bien avec les fonds de carte 
`Stamen`, `OpenStreetMap` ou `Google Maps`. En toute rigueur, si on
désire effectuer certains calculs géométriques (mesurer des surfaces...), il est
nécessaire de re-projeter les données dans un système qui préserve la géométrie
(c'est le cas du Lambert 93). 

Pour avoir une intuition de la localisation des stations, et notamment de la
densité hétérogène de celles-ci, 
on peut afficher les données sur la carte des communes
de la petite couronne. Il s'agit donc d'enrichir la carte
précédente d'une couche supplémentaire, à savoir la localisation
des stations. Au passage, on va utiliser un fond de carte
plus esthétique:


In [ ]:
#| echo: true

fig,ax = plt.subplots(figsize=(10, 10))
stations.sample(200).to_crs(3857).plot(ax = ax, color = 'red', alpha = 0.4, zorder=2)
shp_communes.to_crs(3857).plot(ax = ax, zorder=1, edgecolor = "black", facecolor="none",
                                                           color = None)
ctx.add_basemap(ax, source = ctx.providers.Stamen.Watercolor)
ax.set_axis_off()

In [ ]:
#| echo: false

fig = ax.get_figure()
fig.savefig("featured.png")

Découvrez ci-dessous par étape les différentes lignes de commandes permettant d'afficher cette carte complète,
étape par étape :

:one:
Afficher le nuage de points de 200 stations vélibs prises au hasard


In [ ]:
#| output: hide

fig,ax = plt.subplots(figsize=(10, 10))
stations.sample(200).to_crs(3857).plot(ax = ax, color = 'red', alpha = 0.4, zorder=2)

:two:
Ajouter à cette couche, en-dessous, les contours des communes


In [ ]:
#| output: false
#| echo: true

fig,ax = plt.subplots(figsize=(10, 10))
stations.sample(200).to_crs(3857).plot(ax = ax, color = 'red', alpha = 0.4, zorder=2)
shp_communes.to_crs(3857).plot(ax = ax, zorder=1, edgecolor = "black", facecolor="none",
                                                           color = None)

In [ ]:
#| echo: false
ax.get_figure()

:three:
Ajouter un fond de carte de type *open street map* grâce au package
`contextily`


In [ ]:
#| output: false
#| echo: true

fig,ax = plt.subplots(figsize=(10, 10))
stations.sample(200).to_crs(3857).plot(ax = ax, color = 'red', alpha = 0.4, zorder=2)
shp_communes.to_crs(3857).plot(ax = ax, zorder=1, edgecolor = "black", facecolor="none",
                                                           color = None)
ctx.add_basemap(ax, source = ctx.providers.Stamen.Watercolor)

In [ ]:
#| echo: false
ax.get_figure()

:four:
Il ne reste plus qu'à retirer l'axe des coordonnées, qui n'est pas très 
esthétique.


In [ ]:
#| output: false
#| echo: true

fig,ax = plt.subplots(figsize=(10, 10))
stations.sample(200).to_crs(3857).plot(ax = ax, color = 'red', alpha = 0.4, zorder=2)
shp_communes.to_crs(3857).plot(ax = ax, zorder=1, edgecolor = "black", facecolor="none",
                                                           color = None)
ctx.add_basemap(ax, source = ctx.providers.Stamen.Watercolor)
ax.set_axis_off()
ax

In [ ]:
#| echo: false
ax.get_figure()

*In fine*, on obtient la carte désirée. 


## Opérations sur les attributs

Toutes les opérations possibles sur un objet `Pandas` le sont également
sur un objet `GeoPandas`. Pour manipuler les données, et non la géométrie,
on parlera d'opérations sur les attributs. 

Par exemple, si on désire 
connaître quelques statistiques sur la taille des stations:


In [ ]:
stations.describe()

Pour connaître les plus grands départements de France métropolitaine,
procédons en deux étapes:

1. Récupérons le contour des communes de France métropolitaine dans son ensemble
grâce à `cartiflette`.
Notons qu'on pourrait récupérer directement les contours départementaux mais
pour l'exercice, nous allons le créer nous-mêmes comme agrégation
des contours communaux
(voir [ce notebook `observable`](https://observablehq.com/@linogaliana/cartiflette-demo) pour la méthode plus
légère qui utilise pleinement les fonctionnalités de `cartiflette`). 
2. Calculons la surface (méthode `area` sur un objet `GeoPandas.GeoDataFrame` ramenée en km², attention néamoins au système de projection comme cela est expliqué plus bas)


In [ ]:
#| output: false
from cartiflette.download import get_vectorfile_ign
france = get_vectorfile_ign(
  borders = "COMMUNE",
  field = "metropole",
  source = "EXPRESS-COG-CARTO-TERRITOIRE",
  provider="IGN"
)

In [ ]:
france['surface'] = france.area.div(10**6)

Les plus grands départements s'obtiennent par une agrégation des
surfaces communales :


In [ ]:
france.groupby('INSEE_DEP').sum(numeric_only = True).sort_values('surface', ascending = False)

Si on veut directement les plus
grandes communes de France métropolitaine :


In [ ]:
france.sort_values('surface', ascending = False).head(10)

Lors des étapes d'agrégation, `groupby` ne conserve pas les géométries. Autrement
dit, si on effectue, par exemple, une somme en fonction d'une variable de groupe avec
le combo `groupby(...).sum(...)` , on perd
la dimension géographique. 

Il est néanmoins possible d'aggréger à la fois les géométries et les 
attribus avec la méthode `dissolve`: 


In [ ]:
fig,ax = plt.subplots(figsize=(10, 10))
france.dissolve(by='INSEE_DEP', aggfunc='sum').plot(ax = ax, column = "surface")
ax.set_axis_off()
ax

Pour produire cette carte, il serait néanmoins plus simple de directement
récupérer les fonds officiels des départements plutôt que d'agréger les
contours des communes:


In [ ]:
dep = get_vectorfile_ign(
  borders = "DEPARTEMENT",
  field = "metropole",
  source = "EXPRESS-COG-CARTO-TERRITOIRE",
  provider="IGN")
dep["area"] = dep.area
ax = dep.plot(column = "area")
ax.set_axis_off()

## Opérations sur les géométries


Outre la représentation graphique simplifiée,
sur laquelle nous reviendrons ultérieurement, l'intérêt principal d'utiliser
`GeoPandas` est l'existence de méthodes efficaces pour
manipuler la dimension spatiale. Un certain nombre proviennent du 
package
[`Shapely`](https://shapely.readthedocs.io/en/latest/manual.html#general-attributes-and-methods). 


::: {.cell .markdown}

```{=html}
<div class="alert alert-danger" role="alert">
<i class="fa-solid fa-triangle-exclamation"></i> Warning</h3>
```


Les données sont en système de coordonnées WGS 84 ou pseudo-Mercator (epsg: `4326`) et ne sont pas projetées.
C'est un format approprié lorsqu'il s'agit d'utiliser un fonds
de carte `OpenStreetMap`, `Stamen`, `Google Maps`, etc.

Mais ce n'est pas un
format sur lequel on désire faire des calculs car les distances sont faussées sans utiliser de projection. D'ailleurs, `geopandas` refusera certaines opérations
sur des données dont le crs est `4326`. On reprojete ainsi les données 
dans la projection officielle pour la métropole, le Lambert 93
(epsg: `2154`).


```{=html}
</div>
```

:::

Comme indiqué ci-dessus, nous reprojetons les données
dans le système Lambert 93 qui ne fausse pas les
calculs de distance et d'aires.


In [ ]:
communes = shp_communes.to_crs(2154)
stations = stations.to_crs(2154)

Par exemple, on peut recalculer la taille d'une commune ou d'arrondissement
avec la méthode `area` (et diviser par $10^6$ pour avoir des $km^2$ au lieu
des $m^2$):


In [ ]:
communes['superficie'] = communes.area.div(10**6)
communes.head(3)

Une méthode qu'on utilise régulièrement est `centroid` qui, comme son nom l'indique,
recherche le centroïde de chaque polygone et transforme ainsi des données
surfaciques en données ponctuelles. Par exemple, pour
représenter approximativement les centres des villages de la
Haute-Garonne (31), après avoir téléchargé le fonds de carte adapté,
fera


In [ ]:
communes_31 = s3.download_vectorfile_url_all(
      crs = 4326,
      values = "31",
      borders="COMMUNE",
      vectorfile_format="topojson",
      filter_by="DEPARTEMENT",
      source="EXPRESS-COG-CARTO-TERRITOIRE",
      year=2022)

# on reprojete en 3857 pour le fond de carte
communes_31 = communes_31.to_crs(3857)

# on calcule le centroide
dep_31 = communes_31.copy()
communes_31['geometry'] = communes_31['geometry'].centroid

ax = communes_31.plot(figsize = (10,10), color = 'red', alpha = 0.4, zorder=2)
dep_31.to_crs(3857).plot(ax = ax, zorder=1, edgecolor = "black", facecolor="none",
                                                           color = None)
ctx.add_basemap(ax, source = ctx.providers.Stamen.Toner)
ax.set_axis_off()
ax

# Gérer le système de projection

Précédemment, nous avons appliqué une méthode `to_crs` pour reprojeter
les données dans un système de projection différent de celui du fichier
d'origine :


In [ ]:
communes = communes.to_crs(2154)
stations = stations.to_crs(2154)

Le système de projection est fondamental pour que la dimension
spatiale soit bien interprétée par `Python`. Un mauvais système de représentation
fausse l'appréciation visuelle mais peut aussi entraîner des erreurs dans
les calculs sur la dimension spatiale.
Ce [post](https://www.earthdatascience.org/courses/earth-analytics/spatial-data-r/geographic-vs-projected-coordinate-reference-systems-UTM/) propose de riches éléments sur le
sujet, notamment l'image suivante qui montre bien le principe d'une projection :

![Les différents types de projection](https://www.earthdatascience.org/images/courses/earth-analytics/spatial-data/spatial-projection-transformations-crs.png)

La Terre peut ainsi être représentée de multiples manières, ce qui n'est pas neutre sur la manière de se représenter
certains continents. 
L'Afrique apparaît beaucoup moins vaste qu'elle ne l'est en réalité sur les cartes utilisant
cette projection.
L'une des déformations les mieux connue est celle provoquée par la projection Mercator. 
Le Groënland paraît avoir
la même surface que l'Amérique du Sud. Pourtant, cette dernière est 8 fois 
plus grande. 

Il existe en fait de nombreuses représentations possibles du monde, plus ou moins 
alambiquées. Les projections sont très nombreuses et certaines peuvent avoir une [forme suprenante](https://imgs.xkcd.com/comics/map_projections.png).
Par exemple,
la [projection de Spillhaus](https://storymaps.arcgis.com/stories/756bcae18d304a1eac140f19f4d5cb3d)
propose de centrer la vue sur les océans et non une terre. C'est pour
cette raison qu'on parle parfois de monde tel que vu par les poissons
à son propos. 


![](truesize.png)
*Exemple de reprojection de pays depuis le site [thetruesize.com](https://thetruesize.com/)* 


Concernant la gestion des projections avec `GeoPandas`,
la [documentation officielle](https://geopandas.org/projections.html) est très bien
faite. Elle fournit notamment l'avertissement suivant qu'il est
bon d'avoir en tête:

> Be aware that most of the time you don’t have to set a projection. Data loaded from a reputable source (using the geopandas.read_file() command) should always include projection information. You can see an objects current CRS through the GeoSeries.crs attribute.
> 
> From time to time, however, you may get data that does not include a projection. In this situation, you have to set the CRS so geopandas knows how to interpret the coordinates.

![](https://imgs.xkcd.com/comics/bad_map_projection_south_america.png)
*Image empruntée à XKCD <https://xkcd.com/2256/> qu'on peut également trouver sur <https://blog.chrislansdown.com/2020/01/17/a-great-map-projection-joke/>*

Pour déterminer le système de projection d'une base de données, on peut vérifier l'attribut `crs`:


In [ ]:
communes.crs

Les deux principales méthodes pour définir le système de projection utilisé sont:

* **`df.set_crs`**: cette commande sert à préciser quel est le système de projection utilisé, c'est-à-dire comment les coordonnées *(x,y)* sont reliées à la surface terrestre. **Cette commande ne doit pas être utilisée pour transformer le système de coordonnées, seulement pour le définir**. 
* **`df.to_crs`**: **cette commande sert à projeter les points d'une géométrie dans une autre, c'est-à-dire à recalculer les coordonnées selon un autre système de projection.** 

Dans le cas particulier de production de carte avec un fond `OpenStreetMaps` ou une carte dynamique `leaflet`, il est nécessaire de dé-projeter les données (par exemple à partir du Lambert-93) pour atterrir dans le système non-projeté WGS 84 (code EPSG 4326). Ce site [dédié aux projections géographiques](https://epsg.io/) peut-être utile pour retrouver le système de projection d'un fichier où il n'est pas indiqué. 

La définition du système de projection se fait de la manière suivante (:warning: avant de le faire, se souvenir de l'avertissement !):

~~~python
communes = communes.set_crs(2154)
~~~

Alors que la reprojection (projection Albers : `5070`) s'obtient de la manière suivante :


In [ ]:
shp_region = get_vectorfile_ign(
  borders = "REGION", field = "metropole",
  source = "EXPRESS-COG-CARTO-TERRITOIRE", provider="IGN")

fig,ax = plt.subplots(figsize=(10, 10))
shp_region.to_crs(5070).plot(ax = ax)
ax

In [ ]:
#| echo: false
ax.set_axis_off()
ax
plt.tight_layout(pad=0, h_pad = 0)
plt.show()
#plt.savefig('reproj1.png', bbox_inches='tight')

On le voit, cela modifie totalement la représentation de l'objet dans l'espace.
Clairement, cette projection n'est pas adaptée aux longitudes et latitudes françaises.
C'est normal, il s'agit d'une projection adaptée au continent 
nord-américain (et encore, pas dans son ensemble !).


In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

fig,ax = plt.subplots(figsize=(10, 10))
world[world.continent == "North America"].to_crs(5070).plot(alpha = 0.2, edgecolor = "k", ax = ax)
ax

In [ ]:
#| echo: false

plt.tight_layout(pad=0, h_pad = 0)
plt.show()
#plt.savefig('reproj2.png', bbox_inches='tight')

# Joindre des données

## Joindre des données sur des attributs

Ce type de jointure se fait entre un objet géographique et un
deuxième objet, géographique ou non. A l'exception de la question 
des géométries, il n'y a pas de différence par rapport à `Pandas`. 

La seule différence avec `Pandas` est dans la dimension géographique.
Si on désire conserver la dimension géographique, il faut faire 
attention à faire :

~~~~python
geopandas_object.merge(pandas_object)
~~~~

Si on utilise deux objets géographiques mais ne désire conserver qu'une seule
dimension géographique[^2], on fera

~~~~python
geopandas_object1.merge(geopandas_object2)
~~~~

Seule la géométrie de l'objet de gauche
sera conservée, même si on fait un *right join*. 


[^2]: Il est techniquement possible d'avoir un `DataFrame` comportant plusieurs
géographies. Par exemple une géométrie polygone et une géométrie point
(le centroid). C'est néanmoins souvent compliqué à gérer et donc peu
recommandable.



## Prolongation possible : joindre des données sur dimension géographique

Le chapitre suivant permettra de mettre en oeuvre des 
jointures géographiques.


::: {.cell .markdown}

```{=html}
<div class="alert alert-warning" role="alert">
<h3 class="alert-heading"><i class="fa-solid fa-lightbulb"></i> Hint</h3>
```


Les jointures spatiales peuvent être très gourmandes en ressources (car il peut être nécessaire de croiser toutes les géométries de `x` avec toutes les géométries de `y`). Voici deux conseils qui peuvent vous aider :

- Il est préférable de tester les jointures géographiques sur un petit échantillon de données, pour estimer le temps et les ressources nécessaires à la réalisation de la jointure.
- Il est parfois possible d'écrire une fonction qui réduit la taille du problème. Exemple: vous voulez déterminer dans quelle commune se situe un logement dont vous connaissez les coordonnées et le département; vous pouvez écrire une fonction qui réalise pour chaque département une jointure spatiale entre les logements situés dans ce département et les communes de ce département, puis empiler les 101 tables de sorties.


```{=html}
</div>
```

:::

# Annexe

## Récupération des données depuis data.gouv

Avec cette méthode, les données des limites administratives demandent donc un peu de travail pour être
importées car elles sont zippées (mais c'est un bon exercice !). 

Le code suivant, dont les 
détails apparaîtront plus clairs après la lecture de la partie
*[webscraping](webscraping)* permet de :

1. Télécharger les données avec `requests` dans un dossier temporaire
2. Les dézipper avec le module `zipfile`

La fonction suivante automatise un peu le processus :


In [ ]:
#| eval: false
import requests
import tempfile
import zipfile

url = 'https://www.data.gouv.fr/fr/datasets/r/0e117c06-248f-45e5-8945-0e79d9136165'
temporary_location = tempfile.gettempdir()

def download_unzip(url, dirname = tempfile.gettempdir(), destname = "borders"):
  myfile = requests.get(url)
  open("{}/{}.zip".format(dirname, destname), 'wb').write(myfile.content)
  with zipfile.ZipFile("{}/{}.zip".format(dirname, destname), 'r') as zip_ref:
      zip_ref.extractall(dirname + '/' + destname)

In [ ]:
#| eval: false
#| output: false

download_unzip(url)

In [ ]:
#| eval: false
#| output: false
shp_communes = gpd.read_file(temporary_location + "/borders/communes-20220101.shp")

Ici, les données ne sont pas projetées puisqu'elles sont dans le 
système WSG84 (epsg: 4326) ce qui permet de facilement ajouter
un fonds de carte `Openstreetmap` ou `Stamen` pour rendre une représentation
graphique plus esthétique.
En toute rigueur, pour faire une carte statique d'un pays en particulier,
il faudrait reprojeter les données dans un système de projection adapté à la zone géographique étudiée
(par exemple le Lambert-93 pour la France métropolitaine).

On peut ainsi représenter Paris pour se donner une idée de la nature
du shapefile utilisé :


In [ ]:
#| eval: false
paris = shp_communes.loc[shp_communes['insee'].str.startswith("75")]

fig,ax = plt.subplots(figsize=(10, 10))
paris.plot(ax = ax, alpha=0.5, edgecolor='blue')
ctx.add_basemap(ax, crs = paris.crs.to_string())
ax.set_axis_off()
ax

On voit ainsi que les données pour Paris ne comportent pas d'arrondissement, 
ce qui est limitant pour une analyse focalisée sur Paris. On va donc les
récupérer sur le site d'open data de la ville de Paris et les substituer 
à Paris :


In [ ]:
#| eval: false
#| echo: true

arrondissements = gpd.read_file("https://opendata.paris.fr/explore/dataset/arrondissements/download/?format=geojson&timezone=Europe/Berlin&lang=fr")
arrondissements = arrondissements.rename(columns = {"c_arinsee": "insee"})
arrondissements['insee'] = arrondissements['insee'].astype(str)
shp_communes = shp_communes[~shp_communes.insee.str.startswith("75")].append(arrondissements)

Pour produire la carte, il faudrait faire:


In [ ]:
#| eval: false
paris = shp_communes.loc[shp_communes.insee.str.startswith("75")]

fig,ax = plt.subplots(figsize=(10, 10))

paris.plot(ax = ax, alpha=0.5, edgecolor='k')
ctx.add_basemap(ax, crs = paris.crs.to_string())
ax.set_axis_off()
ax

## Récupération des données depuis le package `pynsee`


Pour connaître les contraintes d'installation du package `pynsee`,
se référer à la partie de cours dédiée à `Pandas`.


In [ ]:
#| output: false
#| eval: false

#le téléchargement des données prend plusieurs minutes
from pynsee.geodata import get_geodata
shp_communes = gpd.GeoDataFrame(
  get_geodata('ADMINEXPRESS-COG-CARTO.LATEST:commune')
)
shp_communes = shp_communes.rename({"insee_com": 'insee'}, axis = 'columns')
#shp_communes = shp_communes.set_crs(3857)